# COMS 4995_002 Deep Learning Assignment 2
Due on Thursday, Feb 27, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Member 1: Michal Porubcin, mp3242

Member 2: Daniel Echlin, dje2126

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import scipy.misc
import glob
import sys
import random
import argparse
import os
import tensorflow as tf
# from tensorflow.contrib.data import Dataset, Iterator

In [2]:
LEARNING_RATE = .001
MAX_ITER = 1000
BATCH_SIZE = 100
DROP_PROB = 0 #0.0-1.0
PERCENT_VALID = 20

In [30]:
#### CNN Code here ####
def train(X_trn, y_trn, X_val, y_val):
    """
    :param X: input samples, each column is a sample
    :param y: labels for input samples, y.shape[0] must equal X.shape[1]
    """

    # Input placeholders, None indicates variable size (for batches)
#     with tf.name_scope('input'):
    X_p = tf.placeholder(tf.float32)
    y_p = tf.placeholder(tf.float32, shape=(None), name='y-input')
#     with tf.name_scope('input_reshape'):
#         image_shaped_input = tf.reshape(x, [-1, 32, 32, 3])
#         tf.summary.image('input', image_shaped_input, 10)

    sess = tf.InteractiveSession()

    def variable_summaries(var):
        """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
                tf.summary.scalar('stddev', stddev)
                tf.summary.scalar('max', tf.reduce_max(var))
                tf.summary.scalar('min', tf.reduce_min(var))
                tf.summary.histogram('histogram', var)

    # get minibatch
    #X_batch, y_batch = get_batch(X, y, batch_size)
    
    # dense layer
    hidden_1 = tf.layers.dense(X_p, units=500, activation=tf.nn.relu)
    variable_summaries(hidden_1.kernel)
    
    # predictions
    y_hat = tf.layers.Dense(inputs=hidden_1,units=10,activation=tf.identity)
    variable_summaries(y_hat.kernel)
    tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    with tf.name_scope('cross_entropy'):
        with tf.name_scope('total'):
            cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_p, logits=y_hat)
    tf.summary.scalar('cross_entropy', cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y_hat, 1), y_p)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to cifar10-hw2 (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test')
    tf.global_variables_initializer().run()

    def feed_dict(isTrain):
        """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
        if isTrain:
            xs, ys = get_batch(X_trn,y_trn,BATCH_SIZE)
            k = 1.0 - DROP_PROB
            return {x: xs, y_: ys, keep_prob: k}
        else:
            xs, ys = get_batch(X_val,y_val,BATCH_SIZE)
            k = 1.0 - DROP_PROB
            return {x: xs, y_: ys, keep_prob: k}

    for i in range(MAX_ITER):
        if i % 10 == 0:  # Record summaries and test-set accuracy
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s: %s' % (i, acc))
        else:  # Record train set summaries, and train
            if i % 100 == 99:  # Record execution stats
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                      feed_dict=feed_dict(True),
                                      run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)
    train_writer.close()
    test_writer.close()
    
def get_batch(self, X, y, batch_size):
    """
    Return minibatch of samples and labels

    :param X, y: samples and corresponding labels
    :parma batch_size: minibatch size
    :returns: (tuple) X_batch, y_batch
    """
    num_samples = X.shape[1]
    indices = np.random.randint(num_samples, size=batch_size)
    X_batch = X[:, indices]
    y_batch = y[indices]
    return X_batch, y_batch

In [4]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def resize_img(img_arr, target_dim=(50, 50)):
    """
    Resizes img represented as numpy array
    """
    return scipy.misc.imresize(img_arr, target_dim)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

# Functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
#         if count > 100:
#             break
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
#     X = tf.map_fn(lambda image: tf.image.per_image_standardization(image), images, tf.int32)
    X = np.column_stack(images)
#     X = images
    return X

def get_train_data(data_root_path, suffix="train"):
    """
    Return X and y
    """
    train_data_path = data_root_path + suffix
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    np.save(filename, y)

In [5]:
# Load train data
data_root_path = './cifar10-hw2/'
# data_root_path = './cifar10-simple/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
# X_centered = (X_train - np.mean(X_train, axis=1)[:, np.newaxis])
# X_div = (X_centered / np.std(X_centered, axis=1)[:, np.newaxis])
# X_train = X_div
print('Train data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


/Users/traes/Documents/15UG/Sem2/DeepLearning/Assignments/HW2/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys


Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Train data loading done


In [6]:
#Load test data
X_test = get_images(data_root_path + 'test')
# X_centered = (X_test - np.mean(X_test, axis=1)[:, np.newaxis])
# X_div = (X_centered / np.std(X_centered, axis=1)[:, np.newaxis])
# X_test = X_div
print('Test data loading done')

/Users/traes/Documents/15UG/Sem2/DeepLearning/Assignments/HW2/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys


Loaded 10000/10000
Test data loading done


In [7]:
#https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

shuffle_in_unison(X_train, y_train)
X_val_index = int(X_train.shape[1]/PERCENT_VALID)
y_val_index = int(len(y_train)/PERCENT_VALID)

X_val, X_trn = X_train[:,:X_val_index], X_train[:,X_val_index:]
y_val, y_trn = y_train[:y_val_index], y_train[y_val_index:]

# trn_dataset = tf.data.Dataset.from_tensor_slices((X_trn,y_trn)).shuffle(buffer_size=len(y_trn)).repeat().batch(BATCH_SIZE)
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val,y_val))#.shuffle().batch(BATCH_SIZE)

# handle = tf.placeholder(tf.string, shape=[])
# #can use either training or validation shapes since identical
# iterator = tf.data.Iterator.from_string_handle(
#     handle, trn_dataset.output_types, trn_dataset.output_shapes)
# next_element = iterator.get_next()

# trn_iter = trn_dataset.make_one_shot_iterator()
# trn_handle = trn_iter.string_handle()

# val_iter = val_dataset.make_one_shot_iterator()
# val_handle = val_iter.string_handle()

# print("ok he11o")

# with tf.Session() as sess:
#     trn_string = sess.run(trn_handle)
#     val_string = sess.run(tst_handle)
#     for i in range(2):
#         print(sess.run(next_element, feed_dict={handle: trn_string}))

#         print(sess.run(next_element, feed_dict={handle: val_string}))

##################

# myiter = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
# train_init_op = myiter.make_initializer(train_dataset)
# test_init_op = myiter.make_initializer(test_dataset)
# features, labels = myiter.get_next() #REMEMBER: lazy eval

# with tf.Session() as sess:
#     for _ in range(2):
#     sess.run(train_init_op) # switch to train dataset
#     for _ in range(EPOCHS):
#         sess.run([features, labels])
#     sess.run(test_init_op) # switch to val dataset
#     print(sess.run([features, labels]))

#Feed Data
# test_features = tf.placeholder(X_test.dtype, X_test.shape)
# test_labels = tf.placeholder(y_test.dtype, y_test.shape)
# test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
# train_features = tf.placeholder(X_train.dtype, X_train.shape)
# train_labels = tf.placeholder(y_train.dtype, y_train.shape)
# train_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))

# iterator = dataset.make_initializable_iterator()
# sess.run(iterator.initializer, feed_dict={features_placeholder: features, labels_placeholder: labels})

In [31]:
#Run CNN
# layer_dimensions = [X_train.shape[0], 500, 500, 500, 10]
train(X_trn,y_trn,X_val,y_val)
#CNN.train(X_train, y_train, iters=2500, alpha=0.0005, batch_size=200)

ValueError: Input 0 of layer dense_2 is incompatible with the layer: its rank is undefined, but the layer requires a defined rank.

In [33]:
#Predict
y_predicted, cache = NN.predict(X_test)
save_predictions('ans1-mp3242', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-mp3242.npy')
print(loaded_y.shape)
loaded_y[:10]

# Write down results here:

Below is our setup:
- Hidden Layers: [1536]
- alpha: .0005, decay factor: .83 (manually set on line 384)
- Batch size: 200
- 2500 iterations

We got about xx% validation accuracy with this run.

Blablabla